Cloning of the wine dataset from a github repository.

In [1]:
! git clone https://github.com/nikitawokurka/Winemodel.git
%cd Winemodel 
%ls

fatal: destination path 'Winemodel' already exists and is not an empty directory.
/content/Winemodel
README.md  winequality-red.csv  winequality-white.csv


# Import TensorFlow and other libraries

Installing Tensorflow 2.0 beta via pip.



In [2]:
pip install tensorflow==2.0.0-beta0

Importing libraries and checking the version of Tensorflow

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

print(tf.__version__)

2.0.0-beta0


# Use Pandas to create a dataframe

Reading the dataset winequality-red.csv and displaying it.

In [4]:
dataframe = pd.read_csv("winequality-red.csv", dtype=float, delimiter=',')
dataframe.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5.0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5.0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5.0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6.0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5.0


Splitting the dataset into a test and validation set.

In [5]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

1023 train examples
256 validation examples
320 test examples


# Create an input pipeline using tf.data



In [0]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('quality')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [0]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

# Create a feature layer

In [0]:
#Definig feature_colums
feature_columns = []

# numeric cols
for header in ["fixed_acidity", "volatile_acidity", "citric_acid", "residual_sugar", "chlorides", "free_sulfur_dioxide", "total_sulfur_dioxide", "density", "pH", "sulphates", "alcohol"]:
  feature_columns.append(feature_column.numeric_column(header))

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)


# Create, compile, and train the mode

Setting the amount of nodes for each layer. Output layer has 10 nodes wich are for the quality betweenn 0 and 10.

Quote from wineDataesciption -
"quality 
between 0 (very bad) and 10 (very excellent). Several data mining methods were applied to model"



In [0]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(64, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(10, activation='softmax')
])

Using the spasparse categorical crossentropy, because we have 10 label classes

linke to API from [SparseCategoricalCrossentropy](hhttps://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy)

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Using only 10 epoch to not overfit our model.



In [11]:
print('Train data set')
model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Train data set
Epoch 1/10


W0614 08:41:38.514212 140589282396032 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


32/32 [==============================] - 2s 54ms/step - loss: 2.2316 - accuracy: 0.2741 - val_loss: 1.5204 - val_accuracy: 0.4180
Epoch 2/10
32/32 [==============================] - 0s 6ms/step - loss: 1.2093 - accuracy: 0.4630 - val_loss: 1.3189 - val_accuracy: 0.4727
Epoch 3/10
32/32 [==============================] - 0s 5ms/step - loss: 1.1614 - accuracy: 0.4903 - val_loss: 1.3398 - val_accuracy: 0.4844
Epoch 4/10
32/32 [==============================] - 0s 5ms/step - loss: 1.1525 - accuracy: 0.4965 - val_loss: 1.3389 - val_accuracy: 0.4688
Epoch 5/10
32/32 [==============================] - 0s 5ms/step - loss: 1.1452 - accuracy: 0.4892 - val_loss: 1.3407 - val_accuracy: 0.4648
Epoch 6/10
32/32 [==============================] - 0s 5ms/step - loss: 1.1403 - accuracy: 0.4865 - val_loss: 1.3441 - val_accuracy: 0.4766
Epoch 7/10
32/32 [==============================] - 0s 5ms/step - loss: 1.1379 - accuracy: 0.4930 - val_loss: 1.3303 - val_accuracy: 0.4727
Epoch 8/10
32/32 [============

Testing model on test data set.

In [12]:
print('Test data set')
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

Test data set
10/10 [==============================] - 0s 2ms/step - loss: 1.0850 - accuracy: 0.4969
Accuracy 0.496875


# Overfitting

Using 200 epochs to see that the accuracy between the train data set and the test data set differ wich is an indicator that the model is overfitting on the train data set.

In [13]:
print('Train data set')
model.fit(train_ds,
          validation_data=val_ds,
          epochs=200)

print('Test data set')
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

Train data set
Epoch 1/200
32/32 [==============================] - 0s 5ms/step - loss: 1.1282 - accuracy: 0.4907 - val_loss: 1.2922 - val_accuracy: 0.4766
Epoch 2/200
32/32 [==============================] - 0s 5ms/step - loss: 1.1240 - accuracy: 0.4917 - val_loss: 1.2723 - val_accuracy: 0.4883
Epoch 3/200
32/32 [==============================] - 0s 5ms/step - loss: 1.1176 - accuracy: 0.5044 - val_loss: 1.2654 - val_accuracy: 0.4727
Epoch 4/200
32/32 [==============================] - 0s 5ms/step - loss: 1.1080 - accuracy: 0.5103 - val_loss: 1.2568 - val_accuracy: 0.4727
Epoch 5/200
32/32 [==============================] - 0s 5ms/step - loss: 1.0988 - accuracy: 0.5132 - val_loss: 1.2599 - val_accuracy: 0.4766
Epoch 6/200
32/32 [==============================] - 0s 5ms/step - loss: 1.0930 - accuracy: 0.5191 - val_loss: 1.2490 - val_accuracy: 0.4648
Epoch 7/200
32/32 [==============================] - 0s 5ms/step - loss: 1.0864 - accuracy: 0.5181 - val_loss: 1.2448 - val_accuracy: 0.488